# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

# imports

In [5]:
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

from decouple import config


# constants

In [ ]:
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

OPEN_ROUTER_API_KEY = config('OPEN_ROUTER_API_KEY')

# set up environment

In [ ]:
openai = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=OPEN_ROUTER_API_KEY,)

# Ollama client (OpenAI-compatible endpoint, as in Day 2)
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')


def messages_for(question: str):
    """Build message list for the LLM (system + user)."""
    system_prompt = "You are a helpful technical tutor who answers questions about python code, software engineering, data science and LLMs."
    user_prompt = "Please give a detailed explanation to the following question: " + question
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]


def explain(question: str, model: str = "gpt") -> str:
    """Get a full explanation from the chosen model. model is 'gpt' or 'llama'."""
    messages = messages_for(question)
    if model == "gpt":
        response = openai.chat.completions.create(model=MODEL_GPT, messages=messages)
        return response.choices[0].message.content or ""
    elif model == "llama":
        response = ollama.chat.completions.create(model=MODEL_LLAMA, messages=messages)
        return response.choices[0].message.content or ""
    else:
        raise ValueError(f"Unknown model: {model}. Use 'gpt' or 'llama'.")


def explain_streaming(question: str, model: str = "gpt"):
    """Stream explanation chunks from the model. Use for GPT in notebook or Gradio."""
    messages = messages_for(question)
    if model != "gpt":
        # Ollama can stream too, but exercise uses non-streaming for Llama; yield full reply
        full = explain(question, model)
        yield full
        return
    stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages, stream=True)
    for chunk in stream:
        delta = chunk.choices[0].delta.content or ""
        if delta:
            yield delta

# here is the question; type over this to ask something new

In [8]:
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

# Call the tool: GPT with streaming

In [9]:
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in explain_streaming(question, model="gpt"):
    response += chunk
    update_display(Markdown(response), display_id=display_handle.display_id)

Sure! Let's break down the code snippet you've provided:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

### 1. Breakdown of the Code

#### 1.1. `books`
This variable is assumed to be a list (or another iterable) of dictionaries, where each dictionary represents a book. Each book dictionary is expected to have an "author" key among potentially other keys.

#### 1.2. `{...}` (Set Comprehension)
The first part of the code is a set comprehension:

```python
{book.get("author") for book in books if book.get("author")}
```

- **Looping through `books`:** `for book in books` iterates over each dictionary in the `books` iterable.
- **Accessing the Author:** `book.get("author")` retrieves the value associated with the "author" key in the dictionary. If the key does not exist, `get` returns `None` by default.
- **Filtering:** The condition `if book.get("author")` ensures that only books with a valid (truthy) author value are considered. This filters out any books where the "author" key is missing or whose value evaluates to `False` (like an empty string).
- **Set Creation:** The set comprehension results in a set of authors. Since sets inherently do not allow duplicate values, any duplicate author names will be stored only once.

#### 1.3. `yield from`
The `yield from` expression is used in a generator function. This allows a generator to yield all values from an iterable (in this case, the set of authors) as if they were directly yielded by the generator function.

### 2. What the Code Does

To summarize, this line of code does the following:

1. **Iterates** through a collection of `books` (assumed to be a list of dictionaries).
2. **Extracts** the authors of those books where an author exists (i.e., the authors are truthy).
3. **Creates** a set of unique author names (avoiding duplicates).
4. **Yields** each unique author name one at a time.

### 3. Why This Code is Useful

- **Memory Efficiency:** By using `yield from`, the authors are generated one at a time, which is beneficial when dealing with a large collection of books. It avoids loading all authors into memory at once.
- **Uniqueness:** Using a set comprehension ensures that only unique author names are produced, making it useful when you want to avoid processing duplicates.
- **Readable and Concise:** This syntax is quite expressive, reading in a clean and straightforward manner while using a single line of code to perform a common pattern of filtering, transformation, and yielding.

### Example Usage

To illustrate the function of the code, consider the following example:

```python
books = [
    {"title": "Book A", "author": "Author 1"},
    {"title": "Book B", "author": "Author 2"},
    {"title": "Book C", "author": "Author 1"},  # Duplicate author
    {"title": "Book D"}  # No author key
]

def unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Example of using the generator
for author in unique_authors(books):
    print(author)
```

This will output:
```
Author 1
Author 2
```
This demonstrates how the code extracts and yields unique author names from the list of books.

# Call the tool: Llama 3.2

In [ ]:
reply = explain(question, model="llama")
display(Markdown(reply))

To understand the line of Python code you've provided, let’s break it down into its components and explain each part step-by-step:

### Breakdown of the Code

1. **`{book.get("author") for book in books if book.get("author")}`**: 
   This part of the code is a **set comprehension**. Here's how it works:
   - `for book in books`: This iterates over each item in the iterable `books`, which is expected to be a collection of dictionaries (each representing a book).
   - `book.get("author")`: This attempts to retrieve the value associated with the key `"author"` from each `book` dictionary. If the key does not exist, it returns `None` instead of throwing a KeyError, which allows for safe access to the data.
   - `if book.get("author")`: This is a conditional filter ensuring that only books with a valid author (where the author is not `None` or an empty string) are considered. Therefore, any books where the author cannot be retrieved (i.e., where `book.get("author")` returns `None` or a falsy value) are excluded from the generated set.
   - The surrounding `{ ... }` denotes that the result of this comprehension is a set, which means that it will contain unique values of authors. This directly avoids any duplicates in authors.

2. **`yield from`**: 
   The `yield from` expression is used in a generator function to yield all values from an iterable. Instead of yielding each value one by one, it allows the generator to encapsulate the logic of iterating through another iterable. 

### Putting it All Together

The entire line of code can be interpreted as follows:
- It creates a set of authors from a list of books, filtering out any entries that do not have a valid author.
- After generating this set, it uses `yield from` to yield each author one at a time, which allows the calling code to iterate over these author values as if they were being produced by the generator function.

### Why Use This Code?

1. **Efficiency**: Using a set comprehension ensures that only unique authors are returned, which can save memory and processing time compared to using a list if there are many duplicate authors.
  
2. **Simplicity**: The use of `yield from` allows the function to maintain a clean structure, making it easier to yield multiple values from an iterable without additional loops or logic.

3. **Clean Iteration**: By yielding authors, this makes it convenient for the caller of the generator to handle each author as it is produced, which is very useful in scenarios where authors need to be processed sequentially.

### Example Scenario

Let's say you have the following list of books:

```python
books = [
    {"title": "Book A", "author": "Alice"},
    {"title": "Book B", "author": "Bob"},
    {"title": "Book C"},  # No author
    {"title": "Book D", "author": "Alice"},  # Duplicate author
    {"title": "Book E", "author": "Charlie"},
]
```

### Code Snippet

Now, if you have this code inside a generator function like so:

```python
def get_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}
```

When you call `get_authors(books)`, it will yield:

- "Alice"
- "Bob"
- "Charlie"

Duplicate authors are eliminated, and books without an author are ignored.

### Conclusion

In summary, the provided code efficiently creates a unique list of authors from a collection of book dictionaries, while yielding them one at a time for further processing. This showcases Python's powerful capabilities with comprehensions and generators.